In [1]:
from experiment_final import NeuralSequenceLabeler
from utils import parseconfig
from utils.conll2003_prepro import process_data
import os

In [6]:
config = parseconfig.parseConfig()
file_dict = {"train":os.path.join(config["raw_path"], "train1.txt"),"dev":os.path.join(config["raw_path"], "valid1.txt"),"test":os.path.join(config["raw_path"], "test1.txt")}
neuralSequenceLabeler =  NeuralSequenceLabeler(config)

Instantiate object of NeuralSequenceLabeler

In [7]:
neuralSequenceLabeler =  NeuralSequenceLabeler(config)

In [8]:
dataset = neuralSequenceLabeler.read_dataset(file_dict,"ontonotes")

split data/raw/benchmark_dataset/raw/train1.txt
split data/raw/benchmark_dataset/raw/test1.txt
split data/raw/benchmark_dataset/raw/valid1.txt
inside read_dataset
train 219554
dev 55044
test 50350


Inititalise metadata & preprocess

In [9]:
train_set,dev_set,test_set,vocab = process_data(dataset,config)
neuralSequenceLabeler.initialize_metadata(vocab)

word embedding shape: [None, None, 300]
chars representation shape: [None, None, 200]
word and chars concatenation shape: [None, None, 500]
rnn output shape: [None, None, 600]
logits shape: [None, None, 9]


/Users/aishwaryaadiga/anaconda2/envs/python3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


params number: 6039690


load the model 

In [10]:
file_dict = {"train": train_set, "dev":dev_set,"test":test_set }

In [11]:
try:
    neuralSequenceLabeler.load_model()
except:
    print("Error loading the model")

in load model....




In [12]:
dataset = neuralSequenceLabeler.read_dataset_helper(file_dict,"ontonotes")

14987
3466
3684


train the model 


In [13]:
neuralSequenceLabeler.train(dataset["train_set"],dataset["dev_data"],dataset["dev_set"],dataset["test_set"])

Start training...
Epoch 1/2:


750/750 [==============================] - 605s - Global Step: 750 - Train Loss: 3.0879   


in predict...


dev dataset -- pre: 76.35, rec: 79.21, FB1: 77.75
 -- new BEST score on test dataset: 77.75
Epoch 2/2:


750/750 [==============================] - 554s - Global Step: 1500 - Train Loss: 0.7879   


in predict...


dev dataset -- pre: 81.73, rec: 83.85, FB1: 82.78
 -- new BEST score on test dataset: 82.78


In [14]:
predictions_formatted = neuralSequenceLabeler.predict(dataset["test_set"],"test")



in predict...


In [15]:
predictions = []
groundTruths = []
words_list = []
for i in range(len(predictions_formatted)):
    predictions_sentence = []
    groundTruths_sentence = []
    words_list_sentence = []
    for j in range(len(predictions_formatted[i])):

        words_list_sentence.append(predictions_formatted[i][j][2])
        predictions_sentence.append(predictions_formatted[i][j][3])

    predictions.append(predictions_sentence)

    words_list.append(words_list_sentence)
for data in dataset["test_set"]:
    for tags,  seq_len in zip(data["tags"], data["seq_len"]):
            tags = [neuralSequenceLabeler.rev_tag_dict[x] for x in tags[:seq_len]]
            groundTruths.append(tags)

In [16]:
print(predictions[2])

['B-PER', 'I-PER']


In [18]:
print(predictions[1])

['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O']


In [19]:
save_path = os.path.join(neuralSequenceLabeler.cfg["checkpoint_path"], "result.txt")
name = "test"
score = neuralSequenceLabeler.evaluate(predictions, groundTruths,words_list, save_path,name)

test dataset -- pre: 81.73, rec: 83.85, FB1: 82.78


Evaluating the predictions 
